## Enhanced record minimum standard compliance of Heritage Places

Enhanced record minimum standard (ERMS) is the minimum standard of data enhancement for heritage places

Import libraries

In [91]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import plotly.express as px

Constants


In [92]:
# Heritage Place Resource Model UUID
uuid_hp = '34cfe992-c2c0-11ea-9026-02e7594ce0a0'
# connection parameters
dbname = "eamena"
user = "eamenar"
password = "eamenar"
host = "52.50.27.140"
port = "5432"
# verbose
verbose = False

Connect the database

In [94]:
try:
    connection = pg.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = connection.cursor()
    if verbose:
        print("Connection established successfully!")
except pg.Error as e:
    print(f"Error: {e}")

## Heritage place selection

Select an HP and get its UUID

In [95]:
selected_hp = 'EAMENA-0500002'
sqll = """
SELECT
      resourceinstanceid AS resourceid
      FROM tiles
      WHERE tiledata -> '%s' -> 'en' ->> 'value' LIKE '%s'
""" % (uuid_hp, selected_hp)
# print(sqll)
cur.execute(sqll)
hpid = cur.fetchone()[0]  
if verbose: 
      print("the UUID of '" + selected_hp + "' is '" + hpid + "'")

## Heritage places field andwith their UUIDs

Read the [output.tsv](https://github.com/eamena-project/eamena-arches-dev/blob/main/dev/data_quality/output.tsv) file with listed UUID linked to fields.
This TSV file is exported automatically ([this GitHub Action](https://github.com/eamena-project/eamena-arches-dev/actions/workflows/update-trigger.yml)) from the [template.xlsx](https://github.com/eamena-project/eamena-arches-dev/blob/main/dev/data_quality/template.xlsx) file, in the same directory

In [96]:
tsv_file = "https://raw.githubusercontent.com/eamena-project/eamena-arches-dev/main/dev/data_quality/output.tsv"
df = pd.read_csv(tsv_file, delimiter='\t')
df = df[["level1", "level2", "level3", "uuid_sql", "Enhanced record minimum standard"]]
df_listed = df.dropna()
print(df_listed.to_markdown())

|    | level1                    | level2                            | level3                                     | uuid_sql                             | Enhanced record minimum standard                                                                                                                             |
|---:|:--------------------------|:----------------------------------|:-------------------------------------------|:-------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  1 | ASSESSMENT SUMMARY        | ASSESSMENT ACTIVITY               | Investigator Role Type                     | d2e1ab96-cc05-11ea-a292-02e7594ce0a0 | Yes                                                                                                                                                          |
|  5 | ASSESSMENT SUMMARY        | ASSESSMENT ACTIVITY    

Select a column (default: `level3`, colud be also `level1`, `level2`) on which the spider plot will be done. Create an empty dataframe, loop over UUIDs to collect data from the HP, and fill the empty dataframe

In [97]:
mylevel = 'level3'
df_erms = df_listed.copy()
df_erms['Enhanced record minimum standard'] = df_erms['Enhanced record minimum standard'].str.contains(r'Yes', case=False, na=False, regex=True).astype(int)
df_erms = df_erms[[mylevel, "Enhanced record minimum standard"]]
df_erms = df_erms.rename(columns={'level3': 'field'})
print(df_erms.to_markdown())

|    | field                                      |   Enhanced record minimum standard |
|---:|:-------------------------------------------|-----------------------------------:|
|  1 | Investigator Role Type                     |                                  1 |
|  5 | GE Imagery Acquisition Date                |                                  1 |
|  8 | Resource Name                              |                                  0 |
|  9 | Name Type                                  |                                  0 |
| 10 | Heritage Place Type                        |                                  0 |
| 11 | General Description Type                   |                                  0 |
| 12 | General Description                        |                                  0 |
| 13 | Heritage Place Function                    |                                  1 |
| 15 | Designation                                |                                  0 |
| 16 | Designation Fr

In [98]:
# empty dataframe
level_values = df_listed[mylevel].unique()
data = {'field': level_values,
        'recorded': np.repeat(0, len(level_values)).tolist()}
df_res = pd.DataFrame(data)
# loop and fill it
for i in df_listed.index:
    if verbose:
        print("read: " + df_listed[mylevel][i] + ' | ' + df_listed['uuid_sql'][i])
    df_field = df_listed[mylevel][i]
    df_field_sql = re.sub(" ", "_", df_field) # rm space
    df_uuid = df_listed['uuid_sql'][i]
    sqll = """
    SELECT value FROM values 
    WHERE valueid::text IN
    (
    SELECT tiledata ->> '%s' AS %s
    FROM tiles 
    WHERE resourceinstanceid::text LIKE '%s'
    AND tiledata -> '%s' IS NOT NULL
    )
    """ % (df_uuid, df_field_sql, hpid, df_uuid)
    if verbose:
        print(sqll)
    cur.execute(sqll)
    outvalue = cur.fetchall()
    if len(outvalue) > 0:
        row_num = df_res[df_res['field'] == df_field].index.tolist()
        df_res.at[row_num[0], 'recorded'] = df_res.loc[row_num[0]]['recorded'] + 1
        if verbose:
            print("recorded values: " + str(outvalue))
if(verbose):
    print(df_res.to_markdown())

## Spider diagram

Show spider diagram with number of fields recorded. If `level3` is selected, the spider plot will also plot the ERMS. 

In [99]:
if mylevel == 'level3':
    colors = {'recorded': 'blue', 'Enhanced record minimum standard': 'red'}
    merged_df = pd.merge(df_res, df_erms, on='field')
    melted_df = pd.melt(merged_df, id_vars=['field'], var_name='Value Set', value_name='Value')
    melted_df.sort_values('Value Set', inplace=True)
    if verbose:
        print(melted_df.to_markdown())
    fig = px.line_polar(melted_df, r='Value', theta='field', color='Value Set',
                        line_close = False, color_discrete_map=colors, title = selected_hp)
    fig.show()
else:
    variable = df_res['field'].tolist()
    value = df_res['nb_of_records'].tolist()
    df = pd.DataFrame(dict(
        value = value,
        variable = variable))
    fig = px.line_polar(df, r = 'value', theta = 'variable', line_close = True)
    fig.show()